In [ ]:
!pip install beautifulsoup4

In [ ]:
pip install pandas


In [ ]:
import os
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [49]:
FOLDER_PATH = "C:\\Users\\stuti\\OneDrive - Symbiosis International University\\Desktop\\HTML_Folder"
BATCH_SIZE = 2500
MAX_WORKERS = 6

In [50]:
# Store results here
data = []

In [51]:
def get_text(soup, query, attr="text"):
    try:
        tag = soup.select_one(query)
        return tag.get(attr) if attr != "text" and tag else tag.get_text(strip=True)
    except:
        return None

In [52]:
def get_table_value(soup, label):
    label = label.lower()

    # STEP 1: Search in known ID-based tables
    table_ids = [
        "technicalSpecifications_section_1",
        "productDetails_techSpec_section_1"
    ]
    for table_id in table_ids:
        table = soup.find("table", id=table_id)
        if table:
            th = table.find("th", string=lambda s: s and label in s.lower())
            if th:
                td = th.find_next_sibling("td")
                if td:
                    return td.get_text(strip=True).replace("\u200e", "")

    # STEP 2: Search in class-based fallback tables
    fallback_tables = soup.find_all("table", class_="prodDetTable")
    for table in fallback_tables:
        for row in table.find_all("tr"):
            cells = row.find_all(["th", "td"])
            if len(cells) >= 2 and label in cells[0].get_text(strip=True).lower():
                return cells[1].get_text(strip=True).replace("\u200e", "")

    # STEP 3: Full fallback – search any generic table
    for table in soup.find_all("table"):
        for row in table.find_all("tr"):
            cells = row.find_all(["th", "td"])
            if len(cells) >= 2:
                header_text = cells[0].get_text(strip=True).lower()
                if label in header_text:
                    return cells[1].get_text(strip=True).replace("\u200e", "")

    return None

In [53]:
def process_file(file_name):
    if not file_name.endswith(".html"):
        return None
    
    print(f"🔄 Scraping {file_name}")

    file_path = os.path.join(FOLDER_PATH, file_name)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f.read(), "html.parser")

            record = {
                "File": file_name,
                "URL": get_text(soup, "link[rel='canonical']", attr="href"),
                "Brand": get_table_value(soup, "Brand"),
                "Product Name": get_text(soup, "#productTitle"),
                "Model Number": get_table_value(soup, "Model Number"),
                "Model Year": get_table_value(soup, "Model Year"),
                "Price": get_text(soup, "span.a-price-whole"),
                "Rating(out of 5)": get_text(soup, "span.a-icon-alt"),
                "Discount (%)": None,

                "Band Colour": get_table_value(soup, "Band Colour"),
                "Band Material": get_table_value(soup, "Band Material"),
                "Band Width": get_table_value(soup, "Band Width"),

                "Case Diameter": get_table_value(soup, "Case Diameter"),
                "Case Material": get_table_value(soup, "Case Material"),
                "Case Thickness": get_table_value(soup, "Case Thickness"),

                "Dial Colour": get_table_value(soup, "Dial Colour"),
                "Crystal Material": get_table_value(soup, "Crystal Material"),
                "Case Shape": get_table_value(soup, "Case Shape"),
                "Movement": get_table_value(soup, "Movement"),
                "Water Resistance Depth": get_table_value(soup, "Water Resistance"),
                "Special Features": get_table_value(soup, "Special Features"),

                "Image": get_text(soup, "#landingImage", attr="data-a-dynamic-image"),
                "ImageURL": get_text(soup, "#landingImage", attr="src")
            }

            discount_tag = soup.find("span", class_=lambda c: c and "savingsPercentage" in c)
            if discount_tag:
                record["Discount (%)"] = discount_tag.get_text(strip=True)

            return record

    except Exception as e:
        print(f"❌ Error processing {file_name}: {e}")
        return None


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

data = []

# Load and limit file list if needed
file_list = sorted(
    [f for f in os.listdir(FOLDER_PATH) if f.endswith(".html")],
    key=lambda x: int(os.path.splitext(x)[0])
)[:]  # ⬅️ change this to run full list or first 25

# Multithreaded execution
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_file, file) for file in file_list]
    for future in as_completed(futures):
        result = future.result()
        if result:
            data.append(result)


In [ ]:
df = pd.DataFrame(data)
df.head(10)

Data Cleaning

In [178]:
df= pd.read_excel(r"C:\Users\stuti\OneDrive - Symbiosis International University\Desktop\HTML_Folder\Final_Watch_Data.xlsx")

In [179]:
df.count()

File                      11226
URL                       11217
Brand                     10876
Product Name              11217
Model Number              10688
Model Year                 1506
Price                     11177
Rating(out of 5)          11217
Discount (%)               7754
Band Colour                9715
Band Material             10489
Band Width                 9443
Case Diameter              9638
Case Material              9595
Case Thickness             9504
Dial Colour                9750
Crystal Material           8723
Case Shape                 9725
Movement                  10569
Water Resistance Depth     8188
Special Features           6864
Image                     11217
ImageURL                  11217
dtype: int64

In [180]:
#delete rows with NaN/blank in URL column and also in the Price Column.
df = df.dropna(subset=["URL"])
df = df.dropna(subset=["Price"])



In [181]:
df.count()

File                      11177
URL                       11177
Brand                     10861
Product Name              11177
Model Number              10678
Model Year                 1504
Price                     11177
Rating(out of 5)          11177
Discount (%)               7754
Band Colour                9703
Band Material             10481
Band Width                 9435
Case Diameter              9624
Case Material              9587
Case Thickness             9493
Dial Colour                9737
Crystal Material           8715
Case Shape                 9711
Movement                  10554
Water Resistance Depth     8181
Special Features           6855
Image                     11177
ImageURL                  11177
dtype: int64

In [182]:
#delete duplicate values with Product Name + Model Number

df = df.drop_duplicates(subset=["Product Name", "Model Number"], keep="first")

In [183]:
df.count()

File                      8672
URL                       8672
Brand                     8479
Product Name              8672
Model Number              8307
Model Year                1131
Price                     8672
Rating(out of 5)          8672
Discount (%)              5549
Band Colour               7865
Band Material             8191
Band Width                7682
Case Diameter             7866
Case Material             7763
Case Thickness            7731
Dial Colour               7897
Crystal Material          7250
Case Shape                7872
Movement                  8256
Water Resistance Depth    6840
Special Features          5538
Image                     8672
ImageURL                  8672
dtype: int64

In [184]:
df["Price"] = (
    df["Price"]
    .str.replace(",", "")                   # Remove commas
    .str.replace(r"\.$", "", regex=True)   # Remove trailing dot if exists
    .astype(float)
    .round(2)
)

df = df[df["Price"] >= 10000] #removing products with price < 10000

In [185]:
df.count()

File                      8136
URL                       8136
Brand                     7967
Product Name              8136
Model Number              7806
Model Year                1053
Price                     8136
Rating(out of 5)          8136
Discount (%)              5434
Band Colour               7405
Band Material             7713
Band Width                7241
Case Diameter             7410
Case Material             7312
Case Thickness            7282
Dial Colour               7433
Crystal Material          6870
Case Shape                7413
Movement                  7767
Water Resistance Depth    6469
Special Features          5182
Image                     8136
ImageURL                  8136
dtype: int64

In [186]:
# df["Rating(out of 5)"].apply(type).value_counts()
# df.head(3)

In [187]:
#adding price band

df["Price Band"] = pd.cut(
    df["Price"],
    bins=[0, 10000, 15000, 25000, 40000, float("inf")],
    labels=["<10K", "10K-15K", "15K-25K", "25K-40K", "40K+"],
    right=False
)


In [188]:
df["Discount (%)"] = (
    df["Discount (%)"]
    .astype(str)
    .str.extract(r"(\d+\.?\d*)")[0]         # extract numeric part
    .replace("", np.nan)
    .astype(float)
    .map(lambda x: f"{int(x)}%" if x.is_integer() else f"{x}%" if pd.notna(x) else np.nan)
)


In [189]:
def normalize_to_mm(value):
    import numpy as np
    import re

    if pd.isna(value) or str(value).strip() == "":
        return np.nan

    text = str(value).strip().lower()
    match = re.search(r"(\d+\.?\d*)", text)
    if not match:
        return np.nan

    num = float(match.group(1))
    if "cm" in text:
        num *= 10  # convert to millimeters

    return f"{int(num)} Millimeters" if num.is_integer() else f"{num} Millimeters"

df["Band Width"] = df["Band Width"].apply(normalize_to_mm)
df["Case Diameter"] = df["Case Diameter"].apply(normalize_to_mm)
df["Case Thickness"] = df["Case Thickness"].apply(normalize_to_mm)


In [190]:
unwanted_keywords = ["pocket watch", "repair tool", "watch bezel", "watch band", "tool", "watch winder", "watch case"]

df = df[~df["Product Name"].str.lower().str.contains('|'.join(unwanted_keywords))]


In [191]:
df.count()

File                      7958
URL                       7958
Brand                     7798
Product Name              7958
Model Number              7635
Model Year                 975
Price                     7958
Rating(out of 5)          7958
Discount (%)              5313
Band Colour               7297
Band Material             7598
Band Width                7165
Case Diameter             7295
Case Material             7201
Case Thickness            7175
Dial Colour               7314
Crystal Material          6763
Case Shape                7296
Movement                  7628
Water Resistance Depth    6425
Special Features          5043
Image                     7958
ImageURL                  7958
Price Band                7958
dtype: int64

In [192]:
##Identifying missing brands for Major Brands

import numpy as np
import re

# Step 1: Define brand keyword mapping
brand_map = {
    "tommy hilfiger": "Tommy Hilfiger",
    "tommy": "Tommy Hilfiger",
    "armani exchange": "Armani Exchange",
    "diesel": "Diesel",
    "fossil": "Fossil",
    "titan": "Titan",
    "casio": "Casio",
    "michael kors": "Michael Kors",
    "maserati": "Maserati",
    "luminox": "Luminox",
    "zeppelin": "Zeppelin",
    "seiko": "Seiko",
    "ted baker": "Ted Baker",
    "invicta": "Invicta",
    "citizen": "Citizen",
    "emporio armani": "Emporio Armani",
    "guess": "Guess",
    "fiece": "Fiece",
    "just cavalli": "Just Cavalli",
    "earnshaw": "Earnshaw",
    "alba": "Alba",
    "daniel wellington": "Daniel Wellington",
    "police": "Police",
    "olevs": "Olevs",
    "ducati": "Ducati",
    "mathey-tissot": "Mathey-Tissot",
    "timex": "Timex",
    "swarovski": "Swarovski",
    "nautica": "Nautica",
    "swiss military hanowa": "Swiss Military Hanowa",
    "lacoste": "Lacoste",
    "boss": "Boss",
    "anne klein": "Anne Klein",
    "calvin klein": "Calvin Klein",
    "pierre cardin": "Pierre Cardin",
    "coach": "Coach",
    "p philip": "P Philip",
    "tag heuer": "Tag Heuer",
    "kenneth cole": "Kenneth Cole",
    "philipp plein": "Philipp Plein",
    "guy laroche": "Guy Laroche",
    "carlos philip": "Carlos Philip",
    "adidas": "Adidas",
    "movado": "Movado",
    "daniel klein": "Daniel Klein",
    "sonata": "Sonata",
    "d1 milano": "D1 Milano",
    "alexandre christie": "Alexandre Christie",
    "santa barbara": "Santa Barbara Polo & Racquet Club",
    "mini cooper": "MINI Cooper",
    "hanowa": "Hanowa",
    "charles-hubert": "Charles-Hubert",
    "gc": "GC"
}

# Step 2: Create helper lowercase column
df["__product_lower__"] = df["Product Name"].str.lower()
df["__brand_match__"] = np.nan

# Step 3: Match known brand keywords in Product Name (exact word match)
for keyword, clean_brand in brand_map.items():
    pattern = rf"\b{re.escape(keyword)}\b"
    mask = df["Brand"].isna() & df["__product_lower__"].str.contains(pattern, regex=True)
    df.loc[mask, "__brand_match__"] = clean_brand

# Step 4: Fill missing Brand from matched keywords
df["Brand"] = df["Brand"].fillna(df["__brand_match__"])

# Step 5: Set remaining missing Brand as "NA"
df["Brand"] = df["Brand"].fillna("NA")

# Step 6: Drop helper columns
df.drop(columns=["__product_lower__", "__brand_match__"], inplace=True)


C:\Users\stuti\AppData\Local\Temp\ipykernel_21568\4133546520.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tommy Hilfiger' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, "__brand_match__"] = clean_brand


In [193]:
def categorize_titan(row):
    brand = str(row["Brand"]).strip().title()
    product = str(row["Product Name"]).strip().title()

    if brand == "Titan":
        if "Xylys" in product:
            return "Titan Xylys"
        elif "Edge" in product:
            return "Titan Edge"
        elif "Raga" in product:
            return "Titan Raga"
        else:
            return "Titan"
    return brand

df["Brand"] = df.apply(categorize_titan, axis=1)


In [194]:
#we are taking product name here because many products donot have a model number, and name is unique


df[df["Brand"].isin(["Titan", "Titan Edge", "Titan Raga", "Titan Xylys"])] \
    .groupby("Brand")["Product Name"] \
    .nunique() \
    .reset_index(name="Unique Product Count") \
    .sort_values(by="Unique Product Count", ascending=False)


,Brand,Unique Product Count
0,Titan,219
1,Titan Edge,63
2,Titan Raga,52
3,Titan Xylys,6


In [ ]:
# # Step 1: Count unique products per brand
# brand_counts = df.groupby("Brand")["Product Name"].nunique()

# # Step 2: Identify brands to keep
# brands_to_keep = ["Titan", "Titan Edge", "Titan Raga", "Titan Xylys"]
# brands_over_50 = brand_counts[brand_counts >= 50].index.tolist()

# # Final list of brands to retain as-is
# final_brands = set(brands_to_keep) | set(brands_over_50)

# # Step 3: Replace other brands with "Others"
# df["Brand"] = df["Brand"].apply(lambda x: x if x in final_brands else "Others")


In [195]:
#Adding a gender column

def infer_gender(product_name):
    name = str(product_name).lower()

    if "couple" in name:
        return "Couple"
    elif any(word in name for word in ["female", "women", "woman", "girl", "ladies", "women's", "girl's"]):
        return "Women"
    elif any(word in name for word in ["male", "man", "men", "boy", "men's", "boy's"]):
        return "Men"
    else:
        return "Unisex"


df["Gender"] = df["Product Name"].apply(infer_gender)

In [196]:
df["Gender"].value_counts()


Gender
Men       5088
Women     2297
Unisex     538
Couple      35
Name: count, dtype: int64

In [197]:
from datetime import datetime

df["As of Date"] = datetime.today().strftime("%Y-%m-%d")

In [199]:
# Sort by file number (e.g., 1.html → 2.html → 3.html)
df = df.sort_values(by="File", key=lambda col: col.map(lambda x: int(os.path.splitext(x)[0])))

output_path = r"C:\Users\stuti\OneDrive - Symbiosis International University\Desktop\HTML_Folder\Final_Watch_Data_Cleaned.xlsx"

df.to_excel(output_path, index=False)

print(f"✅ Data saved to {output_path}")

✅ Data saved to C:\Users\stuti\OneDrive - Symbiosis International University\Desktop\HTML_Folder\Final_Watch_Data_Cleaned.xlsx
